In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser


In [2]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [3]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
pwd

'/notebooks/notebooks'

In [5]:
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# # data = cursor.execute("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]").fetchall()
# rules_df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]", conn)
# # org_df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)
# conn.close()

In [6]:
# Changed Code for audit integration
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# data = cursor.execute("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]").fetchall()
cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), '2_RISK_PERSON',GETDATE(),'Error','RISK_PERSON','PERSON_MASTER',(select max(update_date) 
from FINCORE_DB.Fincore.person_master))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = '2_RISK_PERSON' ",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)
# org_df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)
conn.commit()
conn.close()


In [7]:
job_id

156

In [8]:
%store job_id

Stored 'job_id' (int64)


In [9]:
# # Changed Code for audit integration update after updating risk_person
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()



# cursor.execute(f'''Update HDM_FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),
# JOB_STATUS = 'Success',
#  SOURCE_MAX_DATE =  (select max(update_date) from HDM_FINCORE_DB.Fincore.person_master) 
#  where  JOB_RUN_ID ={job_id}''')


# cursor.commit()
# conn.close()



# # update_query = ('''Update HDM_FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success',
# #  SOURCE_MAX_DATE =  (select max(update_date) from HDM_FINCORE_DB.Fincore.person_master),
# #  NO_OF_RECORD = Null where JOB_RUN_ID = ? ''',job_id)

# # cursor.execute(update_query)
# # conn.close()

In [10]:
rules_df

,RULE_ID,RULE_NO,RULE_DESC,PERSON_TYPE,RISK_TYPE,IS_ACTIVE_FLAG,CREATE_DATE,UPDATE_DATE,SOURCE_DATA_EXISTING_UNDRSTD,DEVELOPMENT_STATUS,CODE_REVIEW_STATUS,DEPENDENCY,WEIGHTAGE,PRIORITY_NO,Query,QUERY_TEMP,RISK_SCORE_STATIC_FLAG
0,3.3.36,T003003036,Multiple deposits made to NGO(s) account(s),NTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,5,4.0,\r\nMerge into [FINCORE_DB].Fincore_Analytic...,None,0.0
1,3.3.37,T003003037,Value of deposits made to NGO(s) account(s) ex...,NTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,5,4.0,\r\n\t\t Merge into [FINCORE_DB]....,None,0.0
2,3.3.38,T003003038,Donation from single entity is greater than pa...,NTR,Transaction Risk,0,2022-02-03,None,None,On Hold,None,None,5,4.0,None,None,0.0
3,3.3.39,T003003039,Multiple donors are contributing more than par...,NTR,Transaction Risk,0,2022-02-03,None,None,On Hold,None,None,5,4.0,None,None,0.0
4,3.3.43,T003003043,Value of transactions destined towards high ri...,CBWTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,8,4.0,\r\n Merge into [FINCORE_DB].Fin...,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2.1.7,O002001007,Number of names/aliases reported across differ...,Organization,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,6,2.0,\r\n MERGE INTO FINCORE_DB.Fincore_Analyti...,None,0.0
222,3.1.8,O003001008,Organization risk given by RE,Organization,KYC profile based Risk,1,2022-02-03,None,None,Testing Completed,None,FUNCTIONAL CLARITY|Need Inputs from Functional...,5,2.0,\r\n MERGE INTO FINCORE_DB.Fincore...,None,1.0
223,3.3.9,T003003009,Cash at ATM_Frequency Risk,TC1_Frequency,Transaction Risk,1,2022-02-03,None,None,Sent for Testing,None,None,7,4.0,\r\nMerge into [FINCORE_DB].Fincore_Analytics...,None,0.0
224,3.3.7,T003003007_D,Cash at ATM_Value Risk (Deposit),TC1_Value_Deposit,Transaction Risk,1,2022-02-03,None,None,Sent for Testing,None,None,7,4.0,\r\nMerge into [FINCORE_DB].Fincore_Analytics...,\r\n\r\nMERGE INTO [FINCORE_DB].[Fincore_Analy...,0.0


In [11]:
rules_df.head()

,RULE_ID,RULE_NO,RULE_DESC,PERSON_TYPE,RISK_TYPE,IS_ACTIVE_FLAG,CREATE_DATE,UPDATE_DATE,SOURCE_DATA_EXISTING_UNDRSTD,DEVELOPMENT_STATUS,CODE_REVIEW_STATUS,DEPENDENCY,WEIGHTAGE,PRIORITY_NO,Query,QUERY_TEMP,RISK_SCORE_STATIC_FLAG
0,3.3.36,T003003036,Multiple deposits made to NGO(s) account(s),NTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,5,4.0,\r\nMerge into [FINCORE_DB].Fincore_Analytic...,None,0.0
1,3.3.37,T003003037,Value of deposits made to NGO(s) account(s) ex...,NTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,5,4.0,\r\n\t\t Merge into [FINCORE_DB]....,None,0.0
2,3.3.38,T003003038,Donation from single entity is greater than pa...,NTR,Transaction Risk,0,2022-02-03,None,None,On Hold,None,None,5,4.0,None,None,0.0
3,3.3.39,T003003039,Multiple donors are contributing more than par...,NTR,Transaction Risk,0,2022-02-03,None,None,On Hold,None,None,5,4.0,None,None,0.0
4,3.3.43,T003003043,Value of transactions destined towards high ri...,CBWTR,Transaction Risk,1,2022-02-03,None,None,Development Completed,None,None,8,4.0,\r\n Merge into [FINCORE_DB].Fin...,None,0.0


In [12]:
rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Individual')]

,RULE_ID,RULE_NO,RULE_DESC,PERSON_TYPE,RISK_TYPE,IS_ACTIVE_FLAG,CREATE_DATE,UPDATE_DATE,SOURCE_DATA_EXISTING_UNDRSTD,DEVELOPMENT_STATUS,CODE_REVIEW_STATUS,DEPENDENCY,WEIGHTAGE,PRIORITY_NO,Query,QUERY_TEMP,RISK_SCORE_STATIC_FLAG
50,2.1.31,P002001031,No of Low Risk CBWTRs filed,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-07-19,None,None,Sent for Testing,None,None,1,2.0,\r\n MERGE INTO FINCORE_DB.Fincore_...,None,0.0
51,2.1.32,P002001032,No of High Risk NTRs filed,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-07-19,None,None,Sent for Testing,None,None,5,2.0,\r\n MERGE INTO FINCORE_DB.Fincore_Ana...,None,0.0
52,2.1.33,P002001033,No of Medium Risk NTRs filed,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-07-19,None,None,Sent for Testing,None,None,3,2.0,\r\n MERGE INTO FINCORE_DB.Fincore...,None,0.0
54,2.1.13,P002001013,Nationality/citizenship country/country of bir...,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,5,2.0,\r\n MERGE INTO FINCORE_DB.Finco...,None,1.0
56,2.1.15,P002001015,Number of Directorships,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,3,2.0,\r\n MERGE INTO FINCORE_DB...,None,0.0
68,2.1.1,P002001001,Number of bank accounts,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-07-19,None,None,Testing Completed,None,None,6,2.0,\r\n MERGE INTO FINCORE_DB.Fincore_Analytics...,None,0.0
69,2.1.2,P002001002,Accounts in too many banks,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,6,2.0,\r\n\r\n MERGE INTO FINCORE_DB.Fi...,None,0.0
72,2.1.4,P002001004,Accounts in high Risk domestic locations,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,8,2.0,\r\n MERGE INTO FINCORE_DB.Finc...,None,0.0
73,2.1.5,P002001005,Addresses in high Risk domestic locations,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,8,2.0,\r\n MERGE INTO FINCORE_DB.Fincore_An...,None,0.0
74,2.1.7,P002001007,Number of names/aliases reported across differ...,Individual,KYC profile/ Inherent risk based High Risk Ind...,1,2022-02-03,None,None,Testing Completed,None,None,6,2.0,\r\n MERGE INTO FINCORE_...,None,0.0


In [13]:
# #This query populates risk_person/org_summary_stg_btchId_reportId


# bid_repid_per_pop = '''
# INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_PERSON_BATCH_REPORT_ID (PERSON_MASTER_ID,BATCH_ID,REPORT_ID)
# SELECT
# A.Batch_Id,A.Report_Id,D.Person_Master_Id
# FROM FINCORE_DB.Fincore_Analytics.Cps_Report_Details AS A
# INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_persondetail AS B
# ON A.Batch_Id = B.batchId AND A.Report_Id = B.reportId
# INNER JOIN FINCORE_DB.Fincore_Analytics.PERSON_CLUSTER AS C ON
# A.Batch_Id = C.Batch_Id AND A.Report_Id = C.Report_Id AND B.id_ = C.Fingate_Person_Id
# INNER JOIN FINCORE_DB.Fincore_Analytics.PERSON_MASTER_CLUSTER_FINGATE_REPORT AS D
# ON C.PERSON_CLUSTER_ID = D.PERSON_CLUSTER_ID and A.Batch_Id = D.batch_Id AND A.Report_Id = D.report_Id AND B.id_ = D.Fingate_Person_Id 
# INNER JOIN FINCORE_DB.Fincore_Analytics.PERSON_MASTER AS E
# ON D.PERSON_MASTER_ID = E.PERSON_MASTER_ID
# WHERE A.Processing_Flag_Cps5 = 1'''

# bid_repid_org_pop = '''
# INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_BATCH_REPORT_ID (ORGANIZATION_MASTER_ID,BATCH_ID,REPORT_ID)
# SELECT
# A.Batch_Id,A.Report_Id,D.ORGANIZATION_Master_Id
# FROM FINCORE_DB.Fincore_Analytics.Cps_Report_Details AS A
# INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_Entitydetail AS B
# ON A.Batch_Id = B.batchId AND A.Report_Id = B.reportId
# INNER JOIN FINCORE_DB.Fincore_Analytics.ORGANIZATION_CLUSTER AS C ON
# A.Batch_Id = C.Batch_Id AND A.Report_Id = C.Report_Id AND B.id_ = C.Fingate_Entity_Id
# INNER JOIN FINCORE_DB.Fincore_Analytics.ORGANIZATION_MASTER_CLUSTER_FINGATE_REPORT AS D
# ON C.ORGANIZATION_CLUSTER_ID = D.ORGANIZATION_CLUSTER_ID and A.Batch_Id = D.batch_Id AND A.Report_Id = D.report_Id AND B.id_ = D.Fingate_Entity_Id 
# INNER JOIN FINCORE_DB.Fincore_Analytics.ORGANIZATION_MASTER AS E
# ON D.ORGANIZATION_MASTER_ID = E.ORGANIZATION_MASTER_ID
# WHERE A.Processing_Flag_Cps5 = 1'''

# bid_repid_org_pop = '''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_BATCH_REPORT_ID (ORGANIZATION_MASTER_ID,BATCH_ID,REPORT_ID)
# SELECT
# A.Batch_Id,A.Report_Id,D.Organization_Master_Id
# FROM FINCORE_DB.FINCORE.Cps_Report_Details AS A
# INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_EntityDetail AS B ON A.Batch_Id = B.batchId AND A.Report_Id = B.reportId
# INNER JOIN FINCORE_DB.FINCORE.Organization_CLUSTER AS C ON A.Batch_Id = C.Batch_Id AND A.Report_Id = C.Report_Id
# INNER JOIN FINCORE_DB.FINCORE.Organization_MASTER_CLUSTER_FINGATE_REPORT AS D ON C.Organization_CLUSTER_ID = D.Organization_CLUSTER_ID
# INNER JOIN FINCORE_DB.FINCORE.Organization_MASTER AS E ON D.Organization_MASTER_ID = E.Organization_MASTER_ID
# WHERE A.Processing_Flag_Cps5 = 1'''

In [14]:
#Queries that populate risk_person_summary_stg





# master_data_pop_per_q1 = '''INSERT INTO [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
# (
# persON_MASTER_ID
# )
# SELECT persON_MASTER_ID
# FROM Fincore.person_Master
# WHERE Is_Active_Flag = 1'''

# master_data_pop_per_q1 = '''SELECT * FROM FINCORE_DB.Fincore.person_master WHERE Is_Active_Flag=1 
# AND Update_Date >
# COALESCE( (select MAX(SOURCE_MAX_DATE) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN
# where JOB_STATUS='Success' and JOB_NAME='2_RISK_PERSON' )
# ,cast('1900-01-01 00:00:00.0000000' as datetime2))'''

master_data_pop_per_q1 = '''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
(
persON_MASTER_ID
)
SELECT persON_MASTER_ID
FROM Fincore.person_Master with (nolock)
WHERE Is_Active_Flag = 1'''

# master_data_pop_per_q1 = '''INSERT INTO [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
# (
# persON_MASTER_ID
# )
# SELECT PERSON_MASTER_ID FROM FINCORE_DB.Fincore.person_master WHERE Is_Active_Flag=1 
# AND Update_Date >
# COALESCE( (select MAX(SOURCE_MAX_DATE) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN
# where JOB_STATUS='Success' and JOB_NAME='2_RISK_PERSON' )
# ,cast('1900-01-01 00:00:00.0000000' as datetime2))
# '''


# master_data_pop_per_q1_new = '''INSERT INTO [HDM_FINCORE_DB].[Fincore_Analytics].[RISK_persON_SUMMARY_STG] 
# (
# BATCH_ID,REPORT_ID,persON_MASTER_ID
# )
# select pmcfr.BATCH_ID,pmcfr.REPORT_ID,pmcfr.persON_MASTER_ID from [FINCORE_DB].[Fincore].person_Master as pm
# inner join [HDM_FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report as pmcfr
# on pm.person_Master_Id = pmcfr.person_Master_Id
# WHERE Is_Active_Flag = 1'''




# master_data_pop_org_q1_new = '''INSERT INTO [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG] 
# (
# BATCH_ID,REPORT_ID,ORGANIZATION_MASTER_ID
# )
# select omcfr.BATCH_ID,omcfr.REPORT_ID,omcfr.ORGANIZATION_MASTER_ID from [FINCORE_DB].[Fincore].Organization_Master as om
# inner join [FINCORE_DB].[Fincore].Organization_Master_Cluster_Fingate_Report as omcfr
# on om.Organization_Master_Id = omcfr.Organization_Master_Id
# WHERE Is_Active_Flag = 1'''





per_pincode_update = '''
UPDATE fincore_db.[Fincore_Analytics].[RISK_person_SUMMARY_STG]
SET PINCODE = B.pin
from 
(select pan as P, pin
from
(
select  pan, try_cast(try_cast(primaryPinCode as int) as decimal(6,0)) as pin,  
rank() over(partition by pan order by legacy_batch_id desc) as orank 
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail] with (nolock)
where pan is not null and primaryPinCode is not null
) T
where orank= 1 

)
B
where pan = B.P'''

per_lat_lon_update = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]
set LATITUDE = p.latitude,
LONGITUDE = p.longitude from (SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[pincode_lat_lon] with (nolock)) p
where [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG].PINCODE = p.PINCODE'''

per_occ_update = '''UPDATE   [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
   SET [OCCUPATION_BUSINESS] = LOB

from 
(
SELECT A.persON_MASTER_ID,
COALESCE( (SELECT TITLE FROM [DATAHUB_DB].[FIUMetaHub].finnet_OccupationType with (nolock) WHERE ID_ = D.occupationId),'Others') as LOB
FROM
fincore_db.Fincore.person_Master A with (nolock)
LEFT JOIN fincore_db.Fincore.person_Master_Cluster_Fingate_Report B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID
LEFT JOIN fincore_db.Fincore.person_Cluster C  with (nolock) ON C.PERSON_CLUSTER_ID = B.PERSON_CLUSTER_ID
LEFT JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_BatchUpload BU with (nolock) ON C.Batch_Id = BU.batchId
LEFT JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_Persondetail D with (nolock) ON BU.Id_ = D.batchId AND C.Report_Id = D.reportId
WHERE A.Is_Active_Flag = 1) B
where [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG].person_master_id = B.Person_master_id'''



# per_occ_update = '''UPDATE [Fincore_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]
#    SET [OCCUPATION_BUSINESS] = LOB

# from 
# (
# select pannumber as P , LOB
# from 
# (
# select pannumber , LOB ,
# rank() over(partition by pannumber order by rcount,LOB desc) as r
# from 
# (
# select pannumber ,  upper(linesOfBusiness) as LOB , count(*) as rcount

# from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_entityDetail]
# where linesOfBusiness is not null 
# 0group by pannumber ,  upper(linesOfBusiness))B ) T
# where t.r = 1) B
# where PAN = B.P'''

# per_trx_update_prod = '''
# UPDATE [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] SET [TRAN_AMNT] = round(total,2)
#   from   
#   (
# select person_master_id as P, sum(total_ctr_amount) as total
# from 
# (
# select PMAM.Person_Master_Id , sum(fgtc2.Amount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo as FGTC2 with (nolock)
# inner join fincore_db.Fincore.account_master AM with (nolock)
# on AM.Account_Number = FGtc2.accountNumber
# inner join fincore_db.Fincore.person_master_account_master as PMAM with (nolock)
# on PMAM.Account_Master_Id = am.Account_Master_Id
# where FGTC2.reportType = cast((select id_ from  DATAHUB_DB.[FIUMetaHub].[finnet_ReportType] with (nolock) where typeName = 'CTR') as varchar)
# group by PMAM.Person_Master_Id 

# union all

# select PMAM.Person_Master_Id , sum(fgtc2.transactionAmount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].[fingate_TsThree]  FGTC2 with (nolock)
# inner join fincore_db.Fincore.account_master AM with (nolock)
# on AM.Account_Number = cast(FGtc2.senderAcNo as varchar)
# inner join fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
# on PMAM.Account_Master_Id = am.Account_Master_Id
# where FGTC2.reportType = cast((select id_ from  DATAHUB_DB.[FIUMetaHub].[finnet_ReportType] with (nolock) where typeName = 'STR') as varchar)
# group by PMAM.Person_Master_Id 

# union all

# select PMAM.Person_Master_Id , sum(fgtc2.transactionAmount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].[fingate_TsThree] as FGTC2 with (nolock)
# inner join fincore_db.Fincore.account_master as AM with (nolock)
# on AM.Account_Number = cast(FGtc2.beneficiaryAcNo as varchar)
# inner join fincore_db.Fincore.person_master_account_master as PMAM with (nolock)
# on PMAM.Account_Master_Id  = am.Account_Master_Id 
# where FGTC2.reportType = cast((select id_ from  DATAHUB_DB.[FIUMetaHub].[finnet_ReportType] with (nolock) where typeName = 'CTR') as varchar)
# group by PMAM.Person_Master_Id 
# ) B
# group by Person_Master_Id) X

# where PERSON_MASTER_ID = X.P'''



master_data_pop_per_q2 = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]

SET PAN = b.pan , 
DIN = b.din ,
full_name = b.full_name,
Father_Name = b.Father_Name,
Date_Of_Birth = b.Date_Of_Birth,
Ckyc_Number = b.Ckyc_Number,
gender = b.gender

from 

(
select Person_Master_Id as entity_id , 
full_name,
Father_Name , 
Date_Of_Birth ,
gender , 
pan , 
din , 
Ckyc_Number
from FINCORE_DB.Fincore.Person_Master with (nolock)
) b

where person_master_id = b.entity_id'''



# per_trx_update = '''UPDATE [FINCORE_DB].[Fincore_analytics].[RISK_PERSON_SUMMARY_STG]
#    SET 
#       [TRAN_AMNT] = round(total,2)
#   from   
#   (
# select person_master_id as P, sum(total_ctr_amount) as total
# from 
# (
# select PMAM.Person_Master_Id , sum(fgtc2.Amount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo as FGTC2
# inner join fincore_db.Fincore.account_master as AM
# on AM.Account_Number = FGtc2.accountNumber
# inner join fincore_db.Fincore.person_master_account_master as PMAM
# on PMAM.Account_Master_Id = am.Account_Master_Id
# where FGTC2.reportType = (select typename from  DATAHUB_DB.[FIUMetaHub].[finnet_ReportType] where typeName = 'CTR')
# group by PMAM.Person_Master_Id 

# union all

# select PMAM.Person_Master_Id , sum(fgtc2.amount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].[fingate_TsOne] as FGTC2
# inner join fincore_db.Fincore.account_master as AM
# on AM.Account_Number = cast(FGtc2.senderAcNo as varchar)
# inner join fincore_db.Fincore.person_master_account_master as PMAM
# on PMAM.Account_Master_Id = am.Account_Master_Id
# where FGTC2.reportType = 'STR'
# group by PMAM.Person_Master_Id 

# union all

# select PMAM.Person_Master_Id , sum(fgtc2.amount) as total_CTR_amount

# from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].[fingate_TsOne] as FGTC2
# inner join fincore_db.Fincore.account_master as AM
# on AM.Account_Number = cast(FGtc2.beneficiaryAcNo as varchar)
# inner join fincore_db.Fincore.person_master_account_master as PMAM
# on PMAM.Account_Master_Id  = am.Account_Master_Id 
# where FGTC2.reportType = 'CTR'
# group by PMAM.Person_Master_Id 
# ) B
# group by Person_Master_Id) X

# where PERSON_MASTER_ID = X.P'''





master_data_pop_per_income = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]
set cbdt_income_range = b.income_range
from (
select pm.Person_Master_Id as entity_id,
case when itr.Gross_Total_Income_Range = 0 then null
when itr.Gross_Total_Income_Range = 1 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 2 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 3 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 4 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 5 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 6 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 7 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 8 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 9 then 10000000 end as Income_range 
from FINCORE_DB.FINCORE.Person_Master pm inner join EXTL_DB.CBDT.CBDT_Itr_Data itr
on itr.Pan =pm.pan inner join   EXTL_DB.CBDT.CBDT_Income_Range   cbdtir
on itr.Gross_Total_Income_Range=cbdtir.Income_Range 
group by pm.Person_Master_Id,cbdtir.Max_Value_Lakh,cbdtir.Min_Value_Lakh,Gross_Total_Income_Range
)b

where person_master_id = b.entity_id'''




In [15]:
per_trx_update_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_CREDIT = round(average,2)
from (


select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
 
 
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id




union 


select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id





)w
group by w.person_master_id
)X

where FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''




per_trx_update_debit = '''
update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_DEBIT = round(average,2)
from (


select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
 
 
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select SENDERACNO ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY SENDERACNO ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
SENDERACNO is not null and SENDERACNO !=''
and transactionType ='DEBIT'
group by SENDERACNO ,transactionDate)t
on t.SENDERACNO=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select senderAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t
on t.senderAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id




union 


select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id


)w
group by w.person_master_id
)X
where 
FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''




peer_grouping_columns = '''Update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set peer_group_amnt = x.coal_val
from (
select PERSON_MASTER_ID, coalesce(cbdt_income_range,TRAN_AMNT_CREDIT,TRAN_AMNT_Debit,0) as coal_val from fincore_db.fincore_analytics.RISK_PERSON_SUMMARY_STG  with(nolock)
)x

where x.PERSON_MASTER_ID = FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.PERSON_MASTER_ID
'''





In [16]:
# ------Mobile Count 
MOBILE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set MOBILE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'M'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''





# ------TELEPHONE Count 
TELEPHONE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TELEPHONE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'T'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ------Email Count 
EMAIL_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set EMAIL_COUNT= k.no_of_emailid from 
(
SELECT B.PERSON_MASTER_ID,  
  COUNT(DISTINCT A.EMAIL_ID) as no_of_emailid       
  FROM  [FINCORE_DB].[Fincore].PERSON_CLUSTER A     with(nolock)
  INNER JOIN  [FINCORE_DB].[Fincore].PERSON_MASTER_CLUSTER_FINGATE_REPORT B   with(nolock)
  ON A.PERSON_CLUSTER_ID = B.PERSON_CLUSTER_ID       
  WHERE A.EMAIL_ID IS NOT NULL    and A.status_active_flag = 1 and    B.is_active_flag =1  
  GROUP BY B.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ------WATCHLIST COUNT 
WATCHLIST_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set WATCHLIST_COUNT= k.watchlist_count from 
(
select  pm.Person_Master_Id,     
               count(w.Agency_Name) as watchlist_count         
               from  FINCORE_DB.Fincore.Person_Master PM       with(nolock)   
               inner join [FINCORE_DB].[Fincore].[Watchlist] W      with(nolock)  
               on PM.PERSON_MASTER_ID = W.Entity_Master_Id            
               where w.Entity_Type = 'I'   and PM.Is_Active_Flag =  1   and W.Is_Active_Flag = 1 group by pm.Person_Master_Id )k
                 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----STR_REPORT_COUNT
STR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set STR_REPORT_COUNT = K.STR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS STR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''






# -----CTR_REPORT_COUNT
CTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CTR_REPORT_COUNT = K.CTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 
                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----NTR_REPORT_COUNT
NTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set NTR_REPORT_COUNT = K.NTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS NTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# -----PTR_REPORT_COUNT
PTR_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set PTR_REPORT_COUNT = K.PTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS PTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'PTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



# ---------------CBWTR_REPORT_COUNT
CBWTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CBWTR_REPORT_COUNT = K.CBWTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CBWTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''






# -----CCR_REPORT_COUNT
CCR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CCR_REPORT_COUNT = K.CCR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CCR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CCR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''







# ----str_amount_debit
str_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_debit = k.str_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
(( PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----str_amount_credit

str_amount_credit =  '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_credit = k.str_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with (nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR' )and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----ctr_amount_debit
ctr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_debit = k.ctr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#  ----ctr_amount_credit

ctr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_credit = k.ctr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



# ----ntr_amount_debit
ntr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_debit = k.ntr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR' )and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----ntr_amount_credit

ntr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_credit = k.ntr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----cbwtr_amount_debit
cbwtr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_debit = k.cbwtr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----cbwtr_amount_credit

cbwtr_amount_credit = ''' update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_credit = k.cbwtr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----ccr_amount_debit
ccr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_debit = k.ccr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----ccr_amount_credit
ccr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_credit = k.ccr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

In [17]:
# Code for batch_id_report_id

# risk_per_b = '''INSERT INTO FINCORE_DB.[Fincore_Analytics].[RISK_PERSON_BATCH_REPORT_ID]
#            ([PERSON_MASTER_ID]
#            ,[BATCH_ID]
#            ,[REPORT_ID]
#            ,[CREATE_DATE]
#            )
# SELECT A.PERSON_MASTER_ID,B.LEGACY_BATCH_ID AS BATCH_ID,B.LEGACY_REPORT_ID AS REPORT_ID,getdate() FROM FINCORE_DB.FINCORE_ANALYTICS.PERSON_MASTER A
# INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_PERSONDETAIL B
# ON A.PAN = B.panNumber  ;'''

# risk_org_b = '''INSERT INTO FINCORE_DB.[Fincore_Analytics].[RISK_ORGANIZATION_BATCH_REPORT_ID]
#            ([ORGANIZATION_MASTER_ID]
#            ,[BATCH_ID]
#            ,[REPORT_ID]
#            ,[CREATE_DATE]
#            )
# SELECT A.ORGANIZATION_MASTER_ID,B.LEGACY_BATCH_ID AS BATCH_ID,B.LEGACY_REPORT_ID AS REPORT_ID,getdate() FROM FINCORE_DB.FINCORE_ANALYTICS.ORGANIZATION_MASTER A
# INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ENTITYDETAIL B
# ON A.PAN = B.panNumber  ;
# '''

In [18]:
# config = ConfigParser()
# config.read(r'config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [19]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [20]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


In [21]:
#Run these queries to populate risk_person_summary_stg table

# [master_data_pop_per_q1,master_data_pop_per_q2,per_pincode_update,per_lat_lon_update,per_occ_update,per_trx_update]
# Making a list of queries to populate the risk_person_summary_stg table
# per_queries = [master_data_pop_per_q1,master_data_pop_per_q2,per_pincode_update,per_lat_lon_update,per_occ_update,per_trx_update]
per_queries = [master_data_pop_per_q1,master_data_pop_per_q2,per_pincode_update,per_occ_update,master_data_pop_per_income,per_trx_update_credit,per_trx_update_debit,peer_grouping_columns,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,STR_REPORT_COUNT,CTR_REPORT_COUNT,NTR_REPORT_COUNT,PTR_COUNT,CBWTR_REPORT_COUNT,CCR_REPORT_COUNT,str_amount_debit,str_amount_credit,ctr_amount_debit,ctr_amount_credit,ntr_amount_debit,ntr_amount_credit,cbwtr_amount_debit,cbwtr_amount_credit,ccr_amount_debit,ccr_amount_credit]
for query in per_queries:
    cursor.execute(query)
    print(namestr(query,globals())[0]," DONE!")
    conn.commit()

cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_Incremental] 
(
persON_MASTER_ID
)
SELECT persON_MASTER_ID
FROM [FINCORE_DB].[Fincore_Analytics].[risk_person_summary_stg] with (nolock)''')

conn.commit()
conn.close()

master_data_pop_per_q1  DONE!
master_data_pop_per_q2  DONE!
per_pincode_update  DONE!
per_occ_update  DONE!
master_data_pop_per_income  DONE!
per_trx_update_credit  DONE!
per_trx_update_debit  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!


In [22]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [23]:
#Populate the risk_person_summary_stg_rules

for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Individual')]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    
conn.close()

('\r\n'
 '            MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_Summary_stg_RULES \r\n'
 '\t\t\tWITH (HOLDLOCK) as target  USING   (   \r\n'
 '\t\t\tSELECT   PMCFR.Person_Master_Id,      \r\n'
 '\t\t\tSUM( 1 ) AS CBWTR_COUNT   \r\n'
 '\t\t\tFROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report '
 'PMCFR \r\n'
 '\t\t\t\r\n'
 '\t\t\tINNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B ON '
 'PMCFR.Report_Type_Id = B.id_     \r\n'
 '\t\t\tinner join FINCORE_DB.Fincore_Analytics.RISK_REPORT rr on '
 'PMCFR.Batch_Id=rr.BATCH_ID and PMCFR.Report_Id=rr.REPORT_ID  \r\n'
 '\t\t\tinner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k on '
 'PMCFR.Person_Master_Id  = k.person_master_id \r\n'
 "\t\t\tWHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR' and "
 'rr.RISK_SCORE<5 GROUP BY PMCFR.person_Master_Id  ) AS source     \r\n'
 '\t\t\tON target.PERSON_MASTER_ID = source.PERSON_master_id and '
 "target.rule_no ='P002001031' WHEN MATCHED THEN  \r\n"
 '\t\